<h2>前言</h2>

在前几周的业余时间，我了解了隐马尔可夫模型，并阅读了一些前人在这个模型上的实践。这些实践都很有启发意义，但是对于我个人而已在以下几点上没有满足我的预期。
<ol>
<li>很多代码没有开源亦或者依赖特定的平台</li>
<li>部分专讲理论，部分专门讲实践，两者都照顾到的似乎很少</li>
</ol>
所以这里我准备先介绍隐马尔科夫模型，接下来在qsq中实现这个模型，并希望能作为一个基础模块来让使用者轻松调用。

<h2>隐马尔科夫模型</h2>
<h3>模型实例</h3>
刘建平Pinard使用了《统计学习方法》中的一个例子来进行解释，我觉得简单易懂。hmm（隐马尔科夫模型）中较为重要的几个部分包括初始状态分布、状态转移概率分布矩阵、观测状态概率矩阵。假设我们有3个盒子，每个盒子里都有红色和白色两种球，这三个盒子里球的数量分别是：

| 盒子 | 1 | 2 | 3 |
| :----: | :----: | :----: | :----: |
| 红球数 | 5 | 4 | 7 |
| 白球数 | 5 | 6 | 3 |

按照下面的方法从盒子里抽球，开始的时候，从第一个盒子抽球的概率是0.2，从第二个盒子抽球的概率是0.4，从第三个盒子抽球的概率是0.4。以这个概率抽一次球后，将球放回。然后从当前盒子转移到下一个盒子进行抽球。规则是：如果当前抽球的盒子是第一个盒子，则以0.5的概率仍然留在第一个盒子继续抽球，以0.2的概率去第二个盒子抽球，以0.3的概率去第三个盒子抽球。如果当前抽球的盒子是第二个盒子，则以0.5的概率仍然留在第二个盒子继续抽球，以0.3的概率去第一个盒子抽球，以0.2的概率去第三个盒子抽球。如果当前抽球的盒子是第三个盒子，则以0.5的概率仍然留在第三个盒子继续抽球，以0.2的概率去第一个盒子抽球，以0.3的概率去第二个盒子抽球。如此下去，直到重复三次，得到一个球的颜色的观测序列:

O = {红，白，红}

这个过程中，观察者只能看到球的颜色序列，却不能看到球是从哪个盒子里取出的。（这里我补充一句，在我们量化的过程中，所想要知道或者猜测的就是当前这个盒子是哪个，也就是当前市场的状态是什么，从而做出相应的策略。）

这个模型中，我们的观察集合是

V = {红，白}， M=2

我们的状态集合是

Q = {盒子1，盒子2，盒子3}，N=3

初始状态分布为

Π = (0.2,0.4,0.4)<sup>T</sup>

状态转移概率分布矩阵为

$$
 \begin{matrix}
   0.5 & 0.2 & 0.3 \\
   0.3 & 0.5 & 0.2 \\
   0.2 & 0.3 & 0.5  
  \end{matrix} 
$$

观测状态概率矩阵为

$$
 \begin{matrix}
   0.5 & 0.5\\
   0.4 & 0.6\\
   0.7 & 0.3  
  \end{matrix} 
$$

总结下这个模型我们需要理解的关键点就是，我们有初始状态分布、状态转移概率分布矩阵、观测状态概率矩阵，在模型实践的过程中，也就是取球的过程中，有一个观测序列O（取出的球的颜色），以及一个隐藏的状态序列（球所在的盒子）。如果我们能够知道现在球所在的盒子，那我们是不是就可以知道下一个是哪个盒子的概率以及取出球的颜色的概率？对应到我们金融市场的情况，大家也就一目了然了吧。

<h3>hmm模型的三个基本问题</h3>
1.评估观察序列概率。给定模型和观测序列，计算该模型下观测序列出现的概率。
2.模型参数学习问题。给定观测序列，估计模型参数，使得该模型下观测序列的条件概率最大。
3.预测问题，给定模型和观测序列，求给定观测序列下最可能出现的状态序列。
我们想利用量化模型来预测股市状态就要解决问题1和3。

<h3>简短总结</h3>
知道我们要解决的问题后，就是如何解决了。hmm模型的解法具体不细讲，因为我自己也不大清楚。有兴趣的朋友可以看下刘建平Pinard博客上的文章介绍，我后面的参考文献中有列出。这里我们需要知道的其实就是hmm怎么用在市场预测中。简单说，应用hmm的目的如下如果只是观测市场，我们只能知道当天的价格、成交量等信息，但是并不知道当前市场处于什么样的状态（牛市、熊市、震荡、反弹等等），在这种情况下我们有两个状态集合，一个可以观察到的状态集合（市场价格成交量状态等）和一个隐藏的状态集合（市场状况）。我们希望能找到一个算法可以根据股市价格成交量状况和马尔科夫假设来预测股市的状况。明白了这一点后，我们就可以化身“调包侠”，愉快地写代码了。

<h2>策略实现</h2>
<h3>增加hmm类</h3>
为了在qsq中加入一个隐马尔可夫策略，我在TacticsQs中加入了QsHmm类。具体看下这个类，主要是实现了以下几个接口。

```python
class QsHmm(object):
    """
    隐马尔可夫策略
    """
    def __init__(self,crypto_df):
        self.components = 6
        self.model = None
        self.iter = 1000
        self.covariance_type = "diag" #convariance_type 包括有"full","spherical","diag","tied"
        self.hidden_states = None
        self.cryptodata = crypto_df #行情数据

    def fit(self, fit_data):
        """
        训练隐马尔可夫模型的接口
        """
        
    def plot_hidden_states(self,cut):
        """
        将隐藏状态和市场状态绘制在一起以显示每个隐藏状态的预测效果
        此处的cut指的是训练时截取数据的偏移，如5day_logreturn，cut为5
        """

    def plot_states_effect(self,cut):
        """
        在每个隐藏状态为True的第二天买入，观察资产增值情况
        此处的cut指的是训练时截取数据的偏移，如5day_logreturn，cut为5
        """
```

具体代码实现，大家可以自行阅读代码。

<h3>训练模型</h3>
<h4>获取数据</h4>

In [4]:
from __future__ import print_function
from __future__ import division

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import os
import sys
# 使用insert 0即只使用github，避免交叉使用了pip安装的abupy，导致的版本不一致问题
sys.path.insert(0, os.path.abspath('../'))

setlocale(LC_NUMERIC, 'English_US')

'English_United States.1252'

In [5]:
import numpy as np

from locale import *
from qsq import QsCrypto, QsHmm

在实际编写策略的过程中我发现，不同的时间维度上，得出的隐藏状态可能差别很大，因为加密货币市场从最初的低价经历了17年的攀升，所以我这里没有分析17年前的数据。实际操练的过程中，大家可以试试看在全部数据上训练的结果，这种情况下会造成隐藏状态的效果很差。在近几年几乎没有任何作用。

In [11]:
bitcoin = QsCrypto('bitcoin')
bitcoin.crypto_df = bitcoin.crypto_df[-700:]
bitcoin.crypto_df

,open,close,high,low,volume,date,MarketCap,pre_close,date_week,p_change
2017-10-30,6114.85,6130.53,6214.99,6040.85,"1,772,150,016",20171030,"102,105,356,519",6153.85,0,-0.379
2017-10-31,6132.02,6468.40,6470.43,6103.33,"2,311,379,968",20171031,"107,743,731,291",6130.53,1,5.511
2017-11-01,6440.97,6767.31,6767.31,6377.88,"2,870,320,128",20171101,"112,735,345,036",6468.40,2,4.621
2017-11-02,6777.77,7078.50,7367.33,6758.72,"4,653,770,240",20171102,"117,933,380,780",6767.31,3,4.598
2017-11-03,7087.53,7207.76,7461.29,7002.94,"3,369,860,096",20171103,"120,097,405,359",7078.50,4,1.826
2017-11-04,7164.48,7379.95,7492.86,7031.28,"2,483,800,064",20171104,"122,978,748,805",7207.76,5,2.389
2017-11-05,7404.52,7407.41,7617.48,7333.19,"2,380,410,112",20171105,"123,449,206,170",7379.95,6,0.372
2017-11-06,7403.22,7022.76,7445.77,7007.31,"3,111,899,904",20171106,"117,050,623,317",7407.41,0,-5.193
2017-11-07,7023.10,7144.38,7253.32,7023.10,"2,326,340,096",20171107,"119,091,456,315",7022.76,1,1.732
2017-11-08,7141.38,7459.69,7776.42,7114.02,"4,602,200,064",20171108,"124,360,767,999",7144.38,2,4.413
